In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import seaborn as sns
import scipy
import datetime
import numpy as np
import time
import copy
import statistics
import os
from scipy.stats import zscore
from scipy.io import savemat
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
from sklearn.ensemble import IsolationForest

Mounted at /content/drive


In [ ]:
#Digitar o caminho para a pasta onde estão os arquivos
caminho = '/content/drive/MyDrive/Teste-IC2025/'
arquivos = os.listdir(caminho)

In [ ]:
arquivos_completos = [os.path.join(caminho, f) for f in os.listdir(caminho) if f.endswith('.pcap')]

In [ ]:
! pip install scapy
from scapy.all import rdpcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00


In [ ]:
#pega todos os arquivos da pasta informada
for j in range(len(arquivos_completos)):

  packets = rdpcap(arquivos_completos[j])
  #packets = rdpcap(arquivos_completos)
  #Variável responsável por separar o nome do arquivo. Usamos ela p/ pegar o valor da corrente
  #Extrair o dado da corrente que está na nome do arquivo. Exemplo: Rot_2000_T_0_Desb_I_043_T021.pcap significa uma corrente média de 0.43A
  #O valor da corrente está no aux[6]
  #Usaremos o nome[0] para pegar o nome do arquivo, q será usado para nomear os arquivos .csv e .mat
  aux=arquivos_completos[j].split(sep='/')
  nome = aux[-1].split(sep='.')
  aux=aux[len(aux)-1].split(sep='_')

  print(nome[0])


  SpeedSetpoint = []
  Current = []
  actualSpeed = []
  actualTorque = []
  SpeedSetpoint = []
  SpeedError = [] #SpeedSetpoint - actualSpeed
  SpeedSetpoint_aux = 0

  MAC_CLP= "28:63:36:fa:6c:70"
  MAC_Servo= "00:1c:06:73:84:64"

  #depois de ler o pacote, percorre todos os seus itens(for i in range(len(packets)))
  for i in range(len(packets)):

  #incio da parte que usa pacotes da rede do MAC_CLP para MAC_Servo (Setpoint de velocidade)
    if packets[i].dst==MAC_Servo:
          b11 = str(hex(packets[i].original[len(packets[i].original)-35]))
          b11_1 = b11.split('0x')
          if len(b11_1[1]) == 1:
              b11='0'+b11_1[1]
          else:
              b11 = b11_1[1]

          b10 = str(hex(packets[i].original[len(packets[i].original)-34]))
          b10_1 = b10.split('0x')
          if len(b10_1[1]) == 1:
              b10='0'+b10_1[1]
          else:
              b10 = b10_1[1]

          SpeedSetpoint_support = '0x'+b11+b10
          SpeedSetpoint_support = (int(SpeedSetpoint_support,16)/int("4000",16))
          SpeedSetpoint.append(SpeedSetpoint_support)

    #inicio da parte que usa pacotes da rede do MAC_Servo para MAC_CLP (Velocidade atual e torque)

    if packets[i].dst==MAC_CLP:
      b11 = str(hex(packets[i].original[len(packets[i].original)-36]))
      b11_1 = b11.split('0x')
      if len(b11_1[1]) == 1:
          b11='0'+b11_1[1]
      else:
          b11 = b11_1[1]

      b10 = str(hex(packets[i].original[len(packets[i].original)-35]))
      b10_1 = b10.split('0x')
      if len(b10_1[1]) == 1:
          b10='0'+b10_1[1]
      else:
          b10 = b10_1[1]

      b7 = str(hex(packets[i].original[len(packets[i].original)-32]))
      b7_1 = b7.split('0x')
      if len(b7_1[1]) == 1:
          b7='0'+b7_1[1]
      else:
          b7 = b7_1[1]

      b6 = str(hex(packets[i].original[len(packets[i].original)-31]))
      b6_1 = b6.split('0x')
      if len(b6_1[1]) == 1:
          b6='0'+b6_1[1]
      else:
          b6 = b6_1[1]

      actualTorque_support = '0x'+b7+b6
      actualTorque_support = (int(actualTorque_support,16)/int("4000",16))
      actualTorque.append(actualTorque_support)
      actualSpeed_support = '0x'+b11+b10
      actualSpeed_support = (int(actualSpeed_support,16)/int("4000",16))
      actualSpeed.append(actualSpeed_support)

      #Current.append(aux[6])


  #Aqui ele já saiu do for que passa por todo o arquivo pcap. Nesta parte fazemos a moda do SpeedSetpoint e atribuimos a uma
  #variável(SpeedSetpoint_aux). Usaremos essa variável para calcular o SpeedError, subtraimos dela o valor do actualSpeed
  moda = stats.mode(SpeedSetpoint, keepdims=False)
  SpeedSetpoint_aux = moda.mode
  print(SpeedSetpoint_aux)

  #Loop responsável por calcular o SpeedError.
  for l in range(len(actualSpeed)):
    SpeedError_support = SpeedSetpoint_aux - actualSpeed[l]  # Para casos com setpoint de velocidade fixos
    SpeedError.append(SpeedError_support)

  #cria os data frame

  df = pd.DataFrame(SpeedError,
                    columns=['Erro'])
  df['Torque'] = pd.DataFrame(actualTorque,
                    columns=['Torque'])
  df['Speed'] = pd.DataFrame(actualSpeed, columns=['Speed'])


  print(df.describe())

  #Remoção de outliers
  #O zscore foi passado 3 vezes para retirar todos os outliers
  z_scores = df[['Speed', 'Torque']].apply(zscore)
  threshold = 5
  filtro = (z_scores.abs() < threshold).all(axis=1)
  df_filtrado1 = df[filtro]
  outliers_removidos1 = df[~filtro]

  z_scores = df_filtrado1[['Speed', 'Torque']].apply(zscore)
  threshold = 5
  filtro = (z_scores.abs() < threshold).all(axis=1)
  df_filtrado2 = df_filtrado1[filtro]
  outliers_removidos2 = df_filtrado1[~filtro]

  z_scores = df_filtrado2[['Speed', 'Torque']].apply(zscore)
  threshold = 5
  filtro = (z_scores.abs() < threshold).all(axis=1)
  df_filtrado3 = df_filtrado2[filtro]
  outliers_removidos3 = df_filtrado2[~filtro]


  print(df_filtrado1.describe())
  print(df_filtrado2.describe())
  print(df_filtrado3.describe())


  #Salva o df_filtrado em um csv
  #colocar aqui a pasta onde serão salvos os arquivos
  caminho_csv = '/content/drive/MyDrive/Teste-IC2025/CSV/'
  df_filtrado3.to_csv(caminho_csv + nome[0] + '.csv',index=False)

  #Salva o df_filtrado em um .mat
  dados_dict = {col: df_filtrado3[col].values for col in df_filtrado3.columns}
  #colocar aqui a pasta onde serão salvos os arquivos
  caminho_mat = '/content/drive/MyDrive/Teste-IC2025/MAT/'
  savemat(caminho_mat + nome[0] + '.mat',  {nome[0]: dados_dict})



Rot_500_T_18_Desb_I_172_T_084
0.16668701171875
               Erro        Torque         Speed
count  61757.000000  61757.000000  61757.000000
mean      -0.018602      0.249538      0.185289
std        0.218469      0.311181      0.218469
min       -3.831726      0.013916      0.011719
25%       -0.000305      0.219666      0.166382
50%        0.000000      0.220642      0.166687
75%        0.000305      0.221741      0.166992
max        0.154968      3.999939      3.998413
               Erro        Torque         Speed
count  61191.000000  61191.000000  61191.000000
mean      -0.000214      0.220977      0.166901
std        0.013527      0.017134      0.013527
min       -0.996277      0.190796      0.139587
25%       -0.000305      0.219666      0.166321
50%        0.000000      0.220642      0.166687
75%        0.000366      0.221741      0.166992
max        0.027100      1.754761      1.162964
               Erro        Torque         Speed
count  61166.000000  61166.000000  61166.

KeyboardInterrupt: 